# Preprocessing
This file handles all preprocessing of the original file 'safe_links_all.gz'

1. drop all non-image files
2. retrieve comments for posts (currently disabled, due to memory leak) 
3. downloading corresponding image, associating with hash of url
4. converting to fasttext format

In [8]:
import sys
import gzip
from collections import namedtuple
import requests
import traceback
import hashlib
import praw
import wget
from praw.models import MoreComments
import logging
import datetime
logging.basicConfig(format='%(asctime)s %(message)s', datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO, stream=sys.stdout)
import os
import os.path
from multiprocessing import Pool
import gensim

2019-01-10 13:08:57 'pattern' package not found; tag filters are not available for English


In [3]:
row = namedtuple('row_raw', ['subreddit', 'submission_title', 'submitted_link', 'comments_link', 'short_name'])
base_path = "/mnt/marcel/"

reddit = praw.Reddit(user_agent='Comment Extraction',
         client_id='BQD4SITRawON4Q', client_secret="noSAX9aSU3VRPowZnhTytXm8sOk",
         )

def retrieve_file(url, filename):
    if not os.path.isfile(filename):
        try:
            wget.download(url, filename, None)
        except:
            pass 

def retrieve_comments(link):
    try:
        comments = reddit.submission(url='https://www.reddit.com'+link).comments.list()
        comments_string = ""
        for comment in comments:
            if not isinstance(comment, MoreComments):
                comments_string += comment.body + " "
        with gzip.open(base_path+'comments/%s.gz' % os.getpid(),'at') as f:
            f.write(str([link, comments_string])+"\n")
    except:
        pass
    
with gzip.open(base_path+'safe_links_all.gz','rt') as f, gzip.open(base_path+'safe_links_imgposts.gz','wt') as fout, Pool(processes=32) as pool:
    for i, line in enumerate(f):
        line = line.replace(',null]', ',""]').replace(',null,', ',"",').replace('[null,', '["",')
        if i%1000000==0:  # total number of lines: 206669601
            logging.info(i//1000000)
        line_list = eval(line)  # convert the str(list) to a list, don't do this in production, as this has the danger of code injection
        r = row._make(line_list)
        submitted_link = r.submitted_link
        if '?' in submitted_link:  # crude way to strip url params
            submitted_link = submitted_link[:r.submitted_link.find('?', 0)]                

        if submitted_link.endswith(".jpg") or submitted_link.endswith(".png") or submitted_link.endswith(".jpeg"):
            imgurlhash = hashlib.sha256(r.submitted_link.encode('utf-8')).hexdigest()
            suffix = ".jpg"
            if submitted_link.endswith(".png"):
                suffix = ".png"
            imgurlhash += suffix

            filename = base_path+"img/"+imgurlhash
            pool.apply_async(retrieve_file, (r.submitted_link, filename))
            #pool.apply_async(retrieve_comments, (r.comments_link, ))

            outlist = list(r)
            outlist.append(imgurlhash)                
            fout.write(str(outlist)+"\n")
            
            #with gzip.open(base_path+'subreddits/%s.gz' % r.subreddit,'at') as f:
            #    f.write(r.submission_title+"\n")

            #outlist.append(comments_string)
            #foutc.write(str(outlist)+"\n")

2019-01-07 00:15:20 0
2019-01-07 00:16:03 1
2019-01-07 00:16:45 2
2019-01-07 00:17:33 3
2019-01-07 00:18:21 4
2019-01-07 00:19:10 5
2019-01-07 00:20:02 6
2019-01-07 00:20:53 7
2019-01-07 00:21:48 8
2019-01-07 00:22:39 9
2019-01-07 00:23:31 10
2019-01-07 00:24:20 11
2019-01-07 00:25:14 12
2019-01-07 00:26:03 13
2019-01-07 00:26:53 14
2019-01-07 00:27:48 15
2019-01-07 00:28:39 16
2019-01-07 00:29:29 17
2019-01-07 00:30:24 18
2019-01-07 00:31:15 19
2019-01-07 00:32:06 20
2019-01-07 00:32:57 21
2019-01-07 00:33:55 22
2019-01-07 00:34:45 23
2019-01-07 00:35:35 24
2019-01-07 00:36:25 25
2019-01-07 00:37:15 26
2019-01-07 00:38:05 27
2019-01-07 00:39:04 28
2019-01-07 00:39:54 29
2019-01-07 00:40:48 30
2019-01-07 00:41:42 31
2019-01-07 00:42:36 32
2019-01-07 00:43:41 33
2019-01-07 00:44:34 34
2019-01-07 00:45:28 35
2019-01-07 00:46:23 36
2019-01-07 00:47:17 37
2019-01-07 00:48:12 38
2019-01-07 00:49:08 39
2019-01-07 00:50:02 40
2019-01-07 00:51:10 41
2019-01-07 00:52:08 42
2019-01-07 00:53:06 4

In [9]:
row = namedtuple('row_raw', ['subreddit', 'submission_title', 'submitted_link', 'comments_link', 'short_name', 'imgurlhash'])

with open(base_path+'imgposts_fasttext.train', "wt") as ft_train, open(base_path+'imgposts_fasttext.valid', "wt") as ft_valid, gzip.open(base_path+'safe_links_imgposts.gz', "rt") as f: 
    for i, line in enumerate(f):
        line_list = eval(line)  # convert the str(list) to a list, don't do this in production, as this has the danger of code injection
        r = row._make(line_list)
        output = "__label__" + r.subreddit + " " + " ".join(gensim.utils.simple_preprocess(r.submission_title)) + "\n"
        if i%10==0:
            ft_valid.write(output)
        else:
            ft_train.write(output)